In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS crypto_catalog.bronze

In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS crypto_catalog.raw;

In [0]:
%sql
CREATE VOLUME IF NOT EXISTS crypto_catalog.raw.crypto_volume;


In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
import requests
import json
import time
import uuid

In [0]:

crypto_schema = StructType([
    StructField("id", StringType(), True),
    StructField("symbol", StringType(), True),
    StructField("name", StringType(), True),
    StructField("current_price", DoubleType(), True),
    StructField("market_cap", DoubleType(), True),
    StructField("total_volume", DoubleType(), True),
    StructField("high_24h", DoubleType(), True),
    StructField("low_24h", DoubleType(), True),
    StructField("price_change_percentage_24h", DoubleType(), True),
    StructField("last_updated", TimestampType(), True)
])



In [0]:

# def fetch_crypto_market_data():
#     try:
#         url = "https://api.coingecko.com/api/v3/coins/markets"
#         params = {
#                 "vs_currency": "usd",
#                 "order": "market_cap_desc",
#                 "per_page": "50",
#                 "page": "1",
#                 "sparkline": "false"
#             }
#         response = requests.get(url, params=params)
#         data = response.json()
#         return data
#     except Exception as e:
#         print("❌ API request failed:", e)
#         return None
    

# landing_path = "/Volumes/crypto_catalog/bronze/crypto_volume/landing"


# data = fetch_crypto_market_data()
# file_path = f"{landing_path}/crypto_{uuid.uuid4()}.json"

# with open(file_path, "w") as f:
#     json.dump(data, f)
    
# print(f"Wrote file: {file_path} | rows: {len(data)}")

    


In [0]:
df = spark.readStream.format("cloudFiles")\
        .option("cloudFiles.format", "json")\
        .option("cloudFiles.schemaLocation", "/Volumes/crypto_catalog/bronze/crypto_volume/checkpoints/bronze/")\
        .option("cloudFiles.schemaEvolutionMode", "rescue")\
        .load("/Volumes/crypto_catalog/bronze/crypto_volume/landing/")

df = df.withColumn("ingestion_timestamp", current_timestamp()) \
       .withColumn("batch_id", lit(str(uuid.uuid4())))

In [0]:
df.writeStream.format("delta").option("checkpointLocation","/Volumes/crypto_catalog/bronze/crypto_volume/checkpoints/bronze/").outputMode("append").trigger(once=True).table("crypto_catalog.bronze.crypto_prices_data")

# df.writeStream.format("delta").outputMode("append").trigger(once=True).option("checkpointLocation", f"/Volumes/workspace/bronze/bronzevolume/{src_value}/checkpoint").option("path", f"/Volumes/workspace/bronze/bronzevolume/{src_value}/data").start()


In [0]:
%sql

-- SELECT * FROM delta.`/Volumes/crypto_catalog/raw/crypto_volume/landing`


SELECT * FROM crypto_catalog.bronze.crypto_prices_data LIMIT 20;
